# Load Data into our DB

In [1]:
import psycopg2
from random_word import RandomWords
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)

r = RandomWords()


In [2]:
languages = ['python', 'javascript', 'java']

In [3]:
conn = psycopg2.connect(database="search",
                        host="localhost",
                        user="ross",
                        password="abc123",
                        port="5432")

In [ ]:
# Add a little "real" data that we can search for

cur = conn.cursor()

for language in languages:
    api_name = f'stripe-{language}-api'
    api_version = 1
    internal_version = 1
    api_base_url = f'www.api.stripe.com'
    company = f'Stripe inc.'
    path_to_zip = '/data/zips'
    programming_language = language
    openapi_url = 'www.google.com'
    openapi_hash = 'abc123'

    # insert data
    cur.execute(f"INSERT INTO metadata(api_name, api_version, internal_version, api_base_url, company, path_to_zip, programming_language, openapi_url, openapi_hash) VALUES('{api_name}', '{api_version}', '{internal_version}', '{api_base_url}', '{company}', '{path_to_zip}', '{programming_language}', '{openapi_url}', '{openapi_hash}')");

conn.commit()
cur.close()

In [ ]:

# insert data in bulks of 10, do this 100000 times for 1 million rows
for i in range(100000):
    print('first loop', i)
    cur = conn.cursor()
    for j in range(50):
        randomBase = r.get_random_word()
        randomBase2 = r.get_random_word()
        api_name = f'{randomBase}-{randomBase2}-api'
        api_version = 1
        internal_version = 1
        api_base_url = f'www.{randomBase}-{randomBase2}-api.com'
        company = f'{randomBase} {randomBase2} inc.'
        path_to_zip = '/data/zips'
        programming_language = random.choice(languages)
        openapi_url = 'www.google.com'
        openapi_hash = 'abc123'

        # insert data
        cur.execute(f"INSERT INTO metadata(api_name, api_version, internal_version, api_base_url, company, path_to_zip, programming_language, openapi_url, openapi_hash) VALUES('{api_name}', '{api_version}', '{internal_version}', '{api_base_url}', '{company}', '{path_to_zip}', '{programming_language}', '{openapi_url}', '{openapi_hash}')");

    print('inserting data')
    conn.commit()
    cur.close()

In [ ]:
cur.close()

# Searching

This would be what our frontend hits when it sends a request to /search

In [4]:
# how many rows do we have?

cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM metadata")
rows = cur.fetchall()
print(rows)
cur.close()

[(9003,)]


In [9]:
searchString = 'stripe jaba'

In [10]:

cur = conn.cursor()

sql = """
SELECT api_name, company, programming_language, rank_ts_search, similarity
FROM
    metadata,
    phraseto_tsquery('english', '{searchString}') query,
    NULLIF(ts_rank(ts, query), 0) rank_ts_search,
    SIMILARITY('{searchString}', search_doc) similarity
WHERE ts @@ query OR similarity > 0
ORDER BY rank_ts_search DESC, similarity DESC
LIMIT 6;
"""

cur.execute(sql)
rows = cur.fetchall()
conn.commit()
cur.close()

pp.pprint(rows)

[   ('stringing-metoxeny-api', 'stringing metoxeny inc.', 'python', None, 0.15),
    ('humour-hamstring-api', 'humour hamstring inc.', 'java', None, 0.13513513),
    (   'semisteel-archaizing-api',
        'semisteel archaizing inc.',
        'javascript',
        None,
        0.13043478),
    ('backstring-marse-api', 'backstring marse inc.', 'python', None, 0.125),
    (   'carbinol-stringpiece-api',
        'carbinol stringpiece inc.',
        'python',
        None,
        0.11363637),
    (   'seabag-intercorrelating-api',
        'seabag intercorrelating inc.',
        'python',
        None,
        0.11111111)]


In [ ]:
conn.close()